In [1]:
import tensorflow as tf
import numpy as np

# 넘파이처럼 텐서플로 사용하기

## 텐서와 연산

In [2]:
tf.constant([[1.,2.,3.], [4.,5.,6.]]) # 행렬

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [3]:
tf.constant(42) # 스칼라

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [4]:
t = tf.constant([[1.,2.,3.], [4.,5.,6.]])
t.shape

TensorShape([2, 3])

In [5]:
t.dtype

tf.float32

In [6]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [7]:
t[...,1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [8]:
t + 10 # == tf.add(t, 10)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [9]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [10]:
t @ tf.transpose(t) # == tf.matmul()

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

## 텐서와 넘파이

* 넘파이 배열 <-> 텐서 가능
* 넘파이 배열에 텐서플로 연산 <-> 텐서에 넘파이 연산 가능  
  
넘파이는 기본으로 64비트 정밀도를 사용하고 텐서플로는 32비트 정밀도를 사용한다.

In [14]:
a = np.array([2.,4.,5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [15]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [16]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [17]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

## 타입 변환

타입이 자동으로 변환되면 사용자가 눈치채지 못할 수 있기 때문에 텐서플로는 어떤 타입 변환도 자동으로 수행하지 않는다. 따라서 호환되지 않는 타입의 텐서로 연산을 실행하면 예외가 발생한다.

In [18]:
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]

필요할 경우 tf.cast() 함수를 사용한다.

In [19]:
t2 = tf.constant(40, dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

## 변수

**tf.Tensor**는 변경이 불가능한 객체, 즉 텐서의 내용을 바꿀 수 없다. 따라서 일반적으로 텐서로는 역전파로 변경되어야 하는 신경망의 가중치를 구현할 수 없다. 또한 시간에 따라 변경되어야 할 다른 파라미터도 있다. 이 것이 **tf.Variable**이 필요한 이유이다.

In [20]:
v = tf.Variable([[1.,2.,3.], [4.,5.,6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

**tf.Variable**은 **tf.Tensor**와 비슷하게 동작하지만, **assign()** 메서드를 사용하여 변수값을 바꿀 수 있다.

In [21]:
v.assign(2*v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [22]:
v[0,1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [24]:
v[:,2].assign([0.,1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [25]:
v.scatter_nd_update(indices=[[0,0], [1,2]], updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

# 사용자 정의 모델과 훈련 알고리즘

## 사용자 정의 손실 함수

회귀 모델을 훈련하는데 훈련 세트에 잡음 데이터가 조금 있는 경우, 이상치 제거 등을 통해 데이터를 수정할 수 있지만 여전히 노이즈 데이터가 남아있을 것이다. 이런 경우, 평균 제곱 오차는 큰 오차에 너무 과한 벌칙을 가하기 때문에 정확하지 않은 모델이 되고, 평균 절대값 오차는 이상치에 관대해서 훈련이 수렴되기까지 시간이 걸리고 정밀하게 훈련되지 안흔다. 이럴 때 평균 제곱 오차 대신 Huber 손실을 사용하면 좋다. Huber 손실은 아직 공식 Keras API에서 지원하지 않기 때문에 직접 구현해야 한다.

In [2]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_los = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [ ]:
model.compile(loss=huber_fn, optimizer="nadam")
model.fit(X_train, y_train)

## 사용자 정의 요소를 가진 모델을 저장하고 로드하기

케라스가 함수 이름을 저장하므로 상요자 정의 손실 함수를 사용하는 모델은 아무 이상 없이 저장된다. 모델을 로드할 때는 함수 이름과 실제 함수를 매핑한 딕셔너리를 전달해야 한다. 좀 더 일반적으로 사용자 정의 객체를 포함한 모델을 로드할 때는 그 이름과 객체를 매핑해야 한다.

In [ ]:
model = keras.models.load_model("my_model_with_a_custom_loss.h5",
                                custom_objects={"huber_fn":huber_fn})

## 활성화 함수, 초기화, 규제, 제한을 커스터마이징하기

In [3]:
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

In [4]:
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2./(shape[0]+shape[1]))
    return tf.random.noraml(shape, stddev=stddev, dtype=dtype)

In [5]:
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01*weights))

In [6]:
def my_positive_weights(weights):
    return tf.where(weights<0., tf.zeros_like(weights), weights)

매개변수는 사용자 정의하려는 함수의 종류에 따라 다르다. 만들어진 사용자 정의 함수는 보통의 함수와 동일하게 사용할 수 있다.

In [ ]:
layer = keras.layers.Dense(30, activation=my_softplus,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)